In [ ]:
# ....carlos do the combined

In [ ]:
ddi_filename = 'ddi_metadata.csv'
fitz_filename = 'fitzpatrick17k.csv'
folder_name = 'raw_data/metadata/date=27-03-2024/'

# Retrieve CSV files from S3
ddi_csv_object = s3.get_object(Bucket=s3_bucket, Key=folder_name + ddi_filename)
fitz_csv_object = s3.get_object(Bucket=s3_bucket, Key=folder_name + fitz_filename)

# Read CSV contents
ddi_csv_content = ddi_csv_object['Body'].read().decode('utf-8')
fitz_csv_content = fitz_csv_object['Body'].read().decode('utf-8')

# Convert CSV contents to DataFrame
df_ddi = pd.read_csv(StringIO(ddi_csv_content))
df_fitz = pd.read_csv(StringIO(fitz_csv_content))

df_merged = pd.DataFrame(index=range(len(df_ddi) + len(df_fitz)), columns=["filename", "skin_tone", "malignant", "path"])
df_merged['filename'] = df_ddi['DDI_file'].tolist() + df_fitz['md5hash'].tolist()
df_merged['filename'] = df_merged['filename'].apply(lambda x: x + ".jpg" if not x.endswith(".png") else x)

df_merged['skin_tone'] = df_ddi['skin_tone'].tolist() + df_fitz['fitzpatrick_scale'].tolist()
df_merged['skin_tone'] = df_merged['skin_tone'].replace([1, 2], 12).replace([3, 4], 34).replace([5, 6], 56)

df_merged['malignant'] = df_ddi['malignant'].tolist() + df_fitz['three_partition_label'].tolist()
df_merged['malignant'] = df_merged['malignant'].replace(['malignant'], True).replace(['non-neoplastic', 'benign'], False)

df_merged['path'] = [folder_name + ddi_filename] * len(df_ddi) + [folder_name + fitz_filename] * len(df_fitz)

df_merged = df_merged[df_merged['skin_tone'] != -1]

folder_prepared = 'prepared/metadata/date=27-03-2024/'

# Convert DataFrame to CSV as bytes
csv_buffer = io.BytesIO()
df_merged.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)  

# Upload the encoded CSV to S3
s3.upload_fileobj(csv_buffer, s3_bucket, folder_prepared + 'merged_metadata.csv')


In [ ]:
# then in the folder modeling the notebook will call the merged_metadata.csv file